In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [2]:
import logging
from oct2py import Oct2Py, get_log
oc = Oct2Py(logger=get_log())
oc.addpath('./src')
oc.logger = get_log('new_log')
oc.logger.setLevel(logging.INFO)

In [3]:
# switch this to cython
def Hbeta(D, beta):
    P = np.exp(-D * beta)
    sumP = np.sum(P)
    H = np.log(sumP) + beta * np.sum(np.multiply(D, P)) / sumP
    P = P / sumP
    return H, P

In [4]:
D = np.random.random(784)
beta = 1 + np.random.randint(64)
%time Ho, Po = oc.Hbeta(D, beta)
%time Hp, Pp = Hbeta(D, beta)
print np.allclose(Ho, Hp) and np.allclose(Po, Pp)

ans =  2
'Hbeta' is a function from the file /Users/kyle/Documents/Learning/Parametric t-SNE/src/Hbeta.m

Hbeta computes the Gaussian kernel values given a vector of
 squared Euclidean distances, and the precision of the Gaussian kernel.
 The function also computes the perplexity of the distribution.


Additional help for built-in functions and operators is
available in the online version of the manual.  Use the command
'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW
at http://www.octave.org and via the help@octave.org
mailing list.
CPU times: user 77.7 ms, sys: 38.8 ms, total: 117 ms
Wall time: 170 ms
CPU times: user 65 µs, sys: 11 µs, total: 76 µs
Wall time: 79.9 µs
True


In [9]:
# switch this to cython
def x2p(X, u=15, tol=1e-4, print_iter=500, max_tries=50):
    # Initialize some variables
    n = X.shape[0]                     # number of instances
    P = np.zeros((n, n))               # empty probability matrix
    beta = np.ones(n)                  # empty precision vector
    logU = np.log(u)                   # log of perplexity (= entropy)
    
    # Compute pairwise distances
    print('Computing pairwise distances...')
    sum_X = np.sum(np.square(X), axis=1)
    # note: translating sum_X' from matlab to numpy means using reshape to add a dimension
    D = sum_X + sum_X.reshape(-1,1) + -2 * X.dot(X.T)

    # Run over all datapoints
    print('Computing P-values...')
    for i in range(n):
        
        if print_iter and i % print_iter == 0:
            print('Computed P-values {} of {} datapoints...'.format(i, n))
        
        # Set minimum and maximum values for precision
        betamin = float('-inf')
        betamax = float('+inf')
        
        # Compute the Gaussian kernel and entropy for the current precision
        indices = np.concatenate((np.arange(0, i), np.arange(i + 1, n)))
        Di = D[i, indices]
        H, thisP = Hbeta(Di, beta[i])
        
        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while abs(Hdiff) > tol and tries < max_tries:
            
            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i]
                if np.isinf(betamax):
                    beta[i] *= 2
                else:
                    beta[i] = (beta[i] + betamax) / 2
            else:
                betamax = beta[i]
                if np.isinf(betamin):
                    beta[i] /= 2
                else:
                    beta[i] = (beta[i] + betamin) / 2
            
            # Recompute the values
            H, thisP = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1
        
        # Set the final row of P
        P[i, indices] = thisP
        
    print('Mean value of sigma: {}'.format(np.mean(np.sqrt(1 / beta))))
    print('Minimum value of sigma: {}'.format(np.min(np.sqrt(1 / beta))))
    print('Maximum value of sigma: {}'.format(np.max(np.sqrt(1 / beta))))
    
    return P, beta

In [10]:
X = np.random.random((1024, 32))
%time Po, betao = oc.x2p(X)
%time Pp, betap = x2p(X)
print np.allclose(Po, Pp) and np.allclose(betao.flatten(), betap.flatten())

Computing pairwise distances...
Computing P-values...
Computed P-values 500 of 1024 datapoints...
Computed P-values 1000 of 1024 datapoints...
Mean value of sigma: 0.47672
Minimum value of sigma: 0.29817
Maximum value of sigma: 0.64675
CPU times: user 992 ms, sys: 469 ms, total: 1.46 s
Wall time: 2.2 s
Computing pairwise distances...
Computing P-values...
Computed P-values 0 of 1024 datapoints...
Computed P-values 500 of 1024 datapoints...
Computed P-values 1000 of 1024 datapoints...
Mean value of sigma: 0.476717700287
Minimum value of sigma: 0.298168280787
Maximum value of sigma: 0.646745154894
CPU times: user 501 ms, sys: 5.93 ms, total: 507 ms
Wall time: 506 ms
True
